In [1]:
import ee
from dotenv import load_dotenv
import os

In [ ]:
ee.Authenticate()

# Load environment variables
load_dotenv()
project_id = os.getenv('GEE_PROJECT_ID')
ee.Initialize(project=project_id)

In [2]:
def get_regions() -> dict:
    """
    Defines the regions of interest with accurate maize growing season dates.

    Returns:
        list: A list of dictionaries, where each dictionary represents a region.
    """
    regions = [
        {
            'name': 'us_cornbelt',
            'roi': ee.Geometry.Rectangle([-96.64, 40.38, -90.14, 43.50]),  # Iowa, USA
            '2023': {'start_date': '2023-04-22', 'end_date': '2023-10-31'},
            '2024': {'start_date': '2024-04-22', 'end_date': '2024-10-31'}
        },
        {
            'name': 'argentina_pampas',
            'roi': ee.Geometry.Rectangle([-65.0, -38.0, -55.0, -30.0]),  # Buenos Aires, Argentina
            '2023': {'start_date': '2022-09-01', 'end_date': '2023-05-31'},
            '2024': {'start_date': '2023-09-01', 'end_date': '2024-05-31'}
        },
        {
            'name': 'brazil_mato_grosso',
            'roi': ee.Geometry.Rectangle([-60.0, -17.0, -50.0, -10.0]),  # Mato Grosso, Brazil
            '2023': {'start_date': '2023-01-01', 'end_date': '2023-07-31'},
            '2024': {'start_date': '2024-01-01', 'end_date': '2024-07-31'}
        },
        {
            'name': 'india_uttar_pradesh',
            'roi': ee.Geometry.Rectangle([77.0, 23.0, 84.0, 30.0]),  # Uttar Pradesh, India
            '2023': {'start_date': '2023-06-01', 'end_date': '2023-11-30'},
            '2024': {'start_date': '2024-06-01', 'end_date': '2024-11-30'}
        }
    ]
    return regions

In [3]:
def create_phenology_mask(collection: ee.ImageCollection, geometry: ee.Geometry.Polygon) -> ee.Image:
    """
    Creates a phenology-based mask to identify areas of agricultural activity.

    Args:
        collection: The Sentinel-2 image collection.
        geometry: The region of interest.

    Returns:
        ee.Image: A single-band image mask where active agricultural areas are 1
                  and other areas are 0.
    """
    try:
        # Calculate a time-series of EVI
        evi_collection = collection.map(lambda image: image.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'BLUE': image.select('B2')
            }).rename('EVI').copyProperties(image, ['system:time_start']))

        # Calculate the min and max EVI
        min_evi = evi_collection.min().rename('min_EVI')
        max_evi = evi_collection.max().rename('max_EVI')

        # Calculate the range (max - min) of EVI for each pixel.
        evi_range = max_evi.subtract(min_evi).rename('EVI_range')

        # Use a dynamic threshold based on the 75th percentile of the EVI range.
        threshold = evi_range.reduceRegion(
            reducer=ee.Reducer.percentile([75]),
            geometry=geometry,
            scale=1000,
            maxPixels=1e10
        ).get('EVI_range').getInfo()

        # Mask 1: Phenology Mask (where EVI range is high)
        phenology_mask = evi_range.gt(threshold)

        # Calculate max NDVI for every pixel over the time series.
        ndvi_collection = collection.map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI'))
        max_ndvi = ndvi_collection.max().rename('max_NDVI')

        # 0.6 is a standard minimum for dense, healthy vegetation.
        vigor_mask = max_ndvi.gt(0.6)

        # Only keep pixels that show BOTH high range AND high peak vigor.
        final_mask = phenology_mask.And(vigor_mask)

        return final_mask.selfMask()

    except Exception as e:
        print(f"Failed to create phenology mask. Using a default mask. Error: {e}")
        # Return a dummy mask of all ones if the process fails
        return ee.Image.constant(1).rename('mask').clip(geometry)


In [4]:
def get_ag_data(geometry: ee.Geometry.Polygon, start_date: str, end_date: str, year) -> ee.ImageCollection:
    """
    Retrieves and processes agricultural-related bands from the Sentinel-2 image collection
    for a given region and time period, and applies a crop-specific mask.

    Args:
        geometry: The region of interest.
        start_date: The start date of the period (YYYY-MM-DD).
        end_date: The end date of the period (YYYY-MM-DD).
        year: The year of the Cropland Data Layer (CDL) to use for masking.
                    Set to None for non-US regions.

    Returns:
        ee.ImageCollection: An ImageCollection with the original bands and new 'NDRE', 
                          'GNDVI', 'NDVI', 'SAVI' and 'EVI' bands, with non-crop pixels masked out.
    """
    # Define a Sentinel-2 image collection, filtering by date and geometry.
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(geometry) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .map(lambda image: image.clip(geometry))

    # Select the relevant bands.
    selected_bands = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'B12']
    s2 = s2.select(selected_bands)

    # Function to calculate various vegetation indices.
    def add_indices(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        evi = image.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'BLUE': image.select('B2')
            }).rename('EVI')
        ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
        gndvi = image.normalizedDifference(['B8', 'B3']).rename('GNDVI')
        savi = image.expression(
            '(1 + L) * (NIR - RED) / (NIR + RED + L)', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'L': 0.5
            }).rename('SAVI')
        return image.addBands(ndvi).addBands(evi).addBands(ndre).addBands(gndvi).addBands(savi)

    # Map the function over the image collection to apply it to all images.
    processed_collection = s2.map(add_indices)

    # Apply the appropriate mask based on the region.
    if year:
        # For the US, we use the specific Cropland Data Layer.
        cdl_image = ee.Image(f'USDA/NASS/CDL/{year}').select('cropland')
        corn_mask = cdl_image.eq(1).selfMask()
    else:
        # For all other regions, we use a phenology-based mask.
        corn_mask = create_phenology_mask(processed_collection, geometry)

    # Apply the corn mask to the entire image collection.
    masked_collection = processed_collection.map(lambda image: image.updateMask(corn_mask))

    return masked_collection


In [5]:
def export_to_drive(image_collection:  ee.ImageCollection, region_name: str, bands_to_export: list) -> None:
    """
    Asynchronously exports the data to your Google Drive.

    Args:
        image_collection: The GEE ImageCollection to export.
        region_name: The name of the region to be used in the output file name.
        bands_to_export: A list of the bands to export.
    """
    # Create a FeatureCollection of random points
    sampled_points = ee.FeatureCollection.randomPoints(
        region=image_collection.first().geometry(),
        points=2000,
        seed=0
    )

    # Map over the image collection and sample each image at the random points.
    mapped_collection = image_collection.map(lambda image: image.select(bands_to_export).sampleRegions(
        collection=sampled_points,
        scale=10,
        geometries=True
    ).map(lambda feature: feature.set('date', image.get('system:time_start'))))

    # Flatten the collection of collections into a single FeatureCollection.
    all_points = mapped_collection.flatten()

    # Export the table as a CSV to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=all_points,
        description=f'GEE_Maize_Data_Export_{region_name}',
        fileNamePrefix=f'GEE_Maize_Data_{region_name}',
        fileFormat='CSV'
    )

    # Start the export task
    task.start()

    print(f"\nGoogle Earth Engine task has been initiated for {region_name}.")


In [ ]:
regions = get_regions()
data_year = 2023  # Change to 2024 to get data for that year.

for region in regions:
    region_name = region['name']
    region_roi = region['roi']
    start_date = region[str(data_year)]['start_date']
    end_date = region[str(data_year)]['end_date']

    # We can only use the CDL for the US.
    crop_data_year = data_year if region_name == 'us_cornbelt' else None

    print(f"\n--- Processing {region_name.upper()} for year {data_year} ---")

    data_collection = get_ag_data(region_roi, start_date, end_date, crop_data_year)

    print(f"Found {data_collection.size().getInfo()} images for the specified region and period.")

    bands_to_export = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'B12', 'NDVI', 'EVI', 'NDRE', 'GNDVI', 'SAVI']

    export_to_drive(data_collection, region_name, bands_to_export)
